In [5]:
# %%capture
# !git clone https://github.com/gem/oq-engine.git

fatal: destination path 'oq-engine' already exists and is not an empty directory.


In [0]:
%%capture
!pip install openquake.engine

In [0]:
%matplotlib inline

from openquake.hazardlib.source import AreaSource
from openquake.hazardlib.mfd import TruncatedGRMFD
from openquake.hazardlib.scalerel import WC1994
from openquake.hazardlib.geo import Point, NodalPlane, Polygon
from openquake.hazardlib.pmf import PMF
from openquake.hazardlib.tom import PoissonTOM
from openquake.hazardlib.calc import stochastic_event_set

import matplotlib.pyplot as plt
import numpy as np

In [0]:
def get_map_projection(src):
    """
    Return map projection specific to source.
    """
    # extract rupture enclosing polygon (considering a buffer of 10 km)
    rup_poly = src.get_rupture_enclosing_polygon(10.)
    min_lon = numpy.min(rup_poly.lons)
    max_lon = numpy.max(rup_poly.lons)
    min_lat = numpy.min(rup_poly.lats)
    max_lat = numpy.max(rup_poly.lats)
    
    # create map projection
    m = Basemap(projection='merc', llcrnrlat=min_lat, urcrnrlat=max_lat,
                llcrnrlon=min_lon, urcrnrlon=max_lon, resolution='l')

    return min_lon, max_lon, min_lat, max_lat, m

In [0]:
def create_ses_legend():
    """
    Create legend for ses plot.
    """
    bb1 = matplotlib.patches.Circle((0, 0), color=MAG_BB_COLORS[0])
    bb2 = matplotlib.patches.Circle((0, 0), color=MAG_BB_COLORS[1])
    bb3 = matplotlib.patches.Circle((0, 0), color=MAG_BB_COLORS[2])
    bb4 = matplotlib.patches.Circle((0, 0), color=MAG_BB_COLORS[3])
    bb5 = matplotlib.patches.Circle((0, 0), color=MAG_BB_COLORS[4])
    matplotlib.pyplot.legend([bb1, bb2, bb3, bb4, bb5],
        ['Mw < 5','5 <= Mw < 6', '6 <= Mw < 7', '7 <= Mw <= 8', 'Mw > 8'], numpoints=1)

In [0]:
# time span for stochastic event set generation
time_span = 1000.

# define area source
src = AreaSource(
    source_id='1',
    name='area',
    tectonic_region_type='Active Shallow Crust',
    mfd=TruncatedGRMFD(min_mag=5., max_mag=6.5, bin_width=0.2, a_val=4.45, b_val=0.98),
    rupture_mesh_spacing=2.,
    magnitude_scaling_relationship=WC1994(),
    rupture_aspect_ratio=1.,
    temporal_occurrence_model=PoissonTOM(time_span),
    upper_seismogenic_depth=2.,
    lower_seismogenic_depth=12.,
    nodal_plane_distribution=PMF([(0.125, NodalPlane(strike=0, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=45, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=90, dip=90, rake=0)),
                                  (0.125, NodalPlane(strike=135, dip=90, rake=0)),
                                  (0.0625, NodalPlane(strike=0, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=45, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=90, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=135, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=180, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=225, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=270, dip=50, rake=90)),
                                  (0.0625, NodalPlane(strike=315, dip=50, rake=90))]),
    hypocenter_distribution=PMF([(1, 7.)]),
    polygon=Polygon([Point(133.5, -22.5), Point(133.5, -23.0), Point(130.75, -23.75), Point(130.75, -24.5),
                     Point(133.5, -26.0), Point(133.5, -27.0), Point(130.75, -27.0), Point(128.977, -25.065),
                     Point(128.425, -23.436), Point(126.082, -23.233), Point(125.669, -22.351), Point(125.4, -20.5),
                     Point(125.75, -20.25), Point(126.7, -21.25), Point(128.5, -21.25), Point(129.25, -20.6),
                     Point(130.0, -20.6), Point(130.9, -22.25), Point(133.0, -22.0), Point(133.5, -22.5)]),
    area_discretization=20.
)

In [0]:
# generate two different stochastic event sets from the area source
numpy.random.seed(123)

# generate event and store them in lists
ses1 = list(stochastic_event_set([src]))
ses2 = list(stochastic_event_set([src]))

In [0]:
# define map projection
min_lon, max_lon, min_lat, max_lat, m = get_map_projection(src)

In [0]:
# plot events for event set 1
for i, ses in enumerate([ses1, ses2]):
    fig = pyplot.figure(figsize=(10, 10), dpi=160)
    
    # m.drawparallels(numpy.arange(min_lat, max_lat, 1.), labels=[True, False, False, True])
    # m.drawmeridians(numpy.arange(min_lon, max_lon, 1.), labels=[True, False, False, True])
    m.drawcoastlines()
    m.drawcountries()
    
    x, y = m(src.polygon.lons, src.polygon.lats)
    m.plot(x, y, color='black', linewidth=2)
    
    for rup in ses:
        strike = rup.surface.get_strike()
        dip = rup.surface.get_dip()
        rake = rup.rake
        x, y = m(rup.hypocenter.longitude, rup.hypocenter.latitude)
        [bb_width] = MAG_BB_WIDTHS[numpy.digitize([rup.mag], MAG_BINS) - 1]
        [bb_color] = MAG_BB_COLORS[numpy.digitize([rup.mag], MAG_BINS) - 1]
        beach = Beach([strike, dip, rake], linewidth=1, xy=(x, y),
                      width=bb_width, zorder=bb_width, facecolor=bb_color)
        pyplot.gca().add_collection(beach)
    
    create_ses_legend()
    pyplot.title('Stochastic Event Set %s for time span of %s years' % (i, time_span), fontsize=20)